In [ ]:
# Copyright 2019 Google LLC
# Copyright 2025 Anonymized Authors

# Licensed under the Apache License, Version 2.0 (the "License"); 
# you may not use this file except in compliance with the License. 
# You may obtain a copy of the License at
# https://www.apache.org/licenses/LICENSE-2.0
"""
This notebook shows the nasbench bias towards operations by setting mutation 
bias to 0,1,2 respectivley. 

Requirements: 

-This notebook requires that numpy and tensorflow be installed within the Python
environment you are running this script in. 

 
"""
import matplotlib.pyplot as plt
import random 
import copy 

import sys; sys.path.append('..')
from utils.nas_utils import load_nasbench, NASBench, NASBenchConstants, SpecOneHot, init_pop
from utils.plotting import plot_all
from IPython.display import clear_output

nasb = load_nasbench()

In [ ]:

def mutate_spec(old_spec, nasbench: NASBench, bias=None, mutation_rate=1.0):
    """Computes a valid mutated spec from the old_spec."""
    while True:
        new_matrix = copy.deepcopy(old_spec.original_matrix)
        new_ops = copy.deepcopy(old_spec.original_ops)

        # In expectation, V edges flipped (note that most end up being pruned).
        edge_mutation_prob = mutation_rate / NASBenchConstants.NUM_VERTICES
        for src in range(0,  NASBenchConstants.NUM_VERTICES - 1):
            for dst in range(src + 1,  NASBenchConstants.NUM_VERTICES):
                if random.random() < edge_mutation_prob:
                    new_matrix[src, dst] = 1 - new_matrix[src, dst]

        # In expectation, one op is resampled.
        op_mutation_prob = mutation_rate / NASBenchConstants.OP_SPOTS
        for ind in range(1,  NASBenchConstants.NUM_VERTICES - 1):
            if random.random() < op_mutation_prob:
                available = [
                    o for o in nasbench.config["available_ops"]
                    if o != new_ops[ind]
                ]
                if bias is None:
                    new_ops[ind] = random.choice(available)
                else:
                    new_ops[ind] = nasbench.config["available_ops"][bias]

        new_spec = SpecOneHot(new_matrix, new_ops)
        if nasbench.is_valid(new_spec):
            return new_spec
        
def mutation(nasbench: NASBench, population, bias=None, mutation_rate=0.5):
    """Mutates each individual of a population."""   

    mutated = []
    for p in population:
        new_spec = mutate_spec(p, nasbench, bias, mutation_rate)
        mutated.append(new_spec)
    return mutated


def run_xevolution(
    nasbench: NASBench,
    bias = 0,
    max_time_budget=5e6,
    p_size=50,
    mutation_rate = 1.0,
):
    """Run a single roll-out of evolution to a fixed time budget."""
    population, times, best_valids, best_tests = init_pop(nasbench, max_time_budget, p_size)

    while True:
        population_only = [p[1] for p in population]
        children = mutation(nasbench, population_only, bias, mutation_rate)

        population = []
        i = 0
        for child in children: 
            # print(i)
            i+=1
            data = nasbench.query(child)
            time_spent, _ = nasbench.get_budget_counters()
            times.append(time_spent)
            # In regularized evolution, we kill the oldest individual.
            population.append((data["validation_accuracy"], child))

            if data["validation_accuracy"] > best_valids[-1]:
                best_valids.append(data["validation_accuracy"])
                best_tests.append(data["test_accuracy"])
            else:
                best_valids.append(best_valids[-1])
                best_tests.append(best_tests[-1])

        if time_spent> max_time_budget:
            break

    return times, best_valids, best_tests


In [ ]:
# comparing with different biases towards peration 0,1,2
o = nasb.config["available_ops"]
exp1 = {
    "data": 
    {
        f"bias {o[0]}" : [[],"Blue"],
        f"bias {o[1]}" : [[],"Dark Orange"],
        f"bias {o[2]}" : [[],"Dark Gray"],
    },
    "config": 
    {
        "budget" : int(1e6),
        "limits" : (0.935, 0.9405),
        "n" : 1000,
        "print_every" : 2,
        "confidence_intervall" : True,
        "pvalue" : 0.05,
        "significant_areas": False,
        "dataset" : "test",
   }
}

budget = exp1["config"]["budget"]
for run in range(exp1["config"]["n"]):
    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, 0, budget, 50)
    exp1["data"][f"bias {o[0]}"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, 1, budget, 50)
    exp1["data"][f"bias {o[1]}"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, 2, budget, 50)
    exp1["data"][f"bias {o[2]}"][0].append((times, best_valid, best_test))


    if (run % exp1["config"]["print_every"] == 0):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        plot_all(exp1, ax)
        plt.show()
        print('Running repeat %d' % (run + 1))

clear_output(wait=True)
fig, ax = plt.subplots()
plot_all(exp1, ax)
plt.savefig(f'A4_ablation_ops_bias.png', dpi=600)
plt.show()